In [ ]:
#Libraries
import pandas as pd
import numpy as np
import glob
import os

In [ ]:
#Set Variables and map to Folder
import_path = 'C:/Users/ianmc/OneDrive/Documents/Work/Data/news_blogs_coronavirus'
os.chdir(import_path)
all_filenames = [i for i in glob.glob('*.{}'.format('JSON'))]
print(all_filenames)

#Read in file(s)
json_data = pd.concat((pd.read_json(f,orient='columns', lines=True) for f in all_filenames), ignore_index=True)
#json_data = pd.read_json(all_filenames[0], lines=True)

In [ ]:
#Drop Unsused Columns to run faster
#unused_columns = ['organizations', 'uuid', 'author', 'url', 'ord_in_thread','title', 'locations', 'entities',
                  'highlightText', 'language','persons', 'text', 'external_links', 'crawled','highlightTitle']
#json_data.drop(unused_columns, axis=1)

#change date columns to show just date
json_data['published'] = pd.to_datetime(json_data['published']).dt.date
json_data['crawled'] = pd.to_datetime(json_data['crawled']).dt.date

In [ ]:
# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

clean_json_data = pd.DataFrame(json_data.text.apply(round1))

# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

clean_json_data = pd.DataFrame(clean_json_data.text.apply(round2))

#add clean text data to orignal file
json_data.insert(column='clean_text', loc=0, value=clean_json_data)

In [ ]:
#More code to come this is just the cleaning part so far -Ian

In [ ]:
export_path = 'C:/Users/ianmc/OneDrive/Documents/Work/Data/news_blogs_coronavirus/'
us_count_per_day.to_csv((export_path), index = True, header = True, encoding='utf-8-sig')